# Ataques contra credenciales de usuario

En esta sección vamos a ver como se pueden realizar ataques contra cuentas de usuario y contraseña para intentar obtener unas credenciales válidas que o bien nos faciliten el acceso a un sistema remoto o bien que nos proporcionen acceso a una cuenta con más privilegios en el sistema o servicio que estemos auditando.

### Metodología

Se recomienda seguir usando [cherrytree](https://www.giuspen.com/cherrytree/) para ir guardando las evidencias.
1. <a style="color:#000;text-decoration:none;cursor:pointer" href="#intro_cuentas">Breve introducción informal a los ataques contra credenciales de usuario</a>
  * Tipos de ataques
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#online">Ataques contra servicios online de forma activa</a>
  * Ataque de diccionario contra SSH usando patator
  * Ataque de diccionario contra formularios web usando Burp Suite
<br><br>
3. <a style="color:#000;text-decoration:none;cursor:pointer" href="#offline">Ataques offline - Cracking de contraseñas</a>
<br><br>

### Referencias usadas

<a name="intro_cuentas"></a>

### Breve introducción informal a los ataques contra credenciales de usuario

Como las credenciales de usuario nos permiten acceder a un sistema, suelen ser un objetivo potencial para todas aquellas personas que deseen tener acceso, autorizado o no, al sistema. Por eso es importante que las credenciales de acceso y, específicamente, las contraseñas sean robustas. Es decir que, entre otros requisitos, cumplan:

  * Que combinen letras mayúsculas y minúsculas, números y símbolos como por ejemplo: .,:@/\|-_+\*
  * Tener una longitud mínima de 24 caracteres unicode
  * Tener una política de cambio de contraseña cada cierto tiempo
  * Asegurarse que no se utiliza una contraseña que ya haya sido utilizada anteriormente
  * Que sea una secuencia lo más aleatoria posible
  * Si no se utilizase una secuencia aleatoria -lo más recomendable-, pueden seguirse estrategias como utilizar la segunda letra de cada palabra de una frase intercalando números, letras mayúsculas,  minúsculas y símbolos.

#### Tipos de ataques

Existen distintos tipos de ataques que podemos realizar para conseguir las credenciales de un usuario. En muchas ocasiones, el usuario lo hemos conseguido en la fase de recopilación de información y sólo nos queda conseguir la contraseña. El temario especifíca 4 tipos de ataques que podemos realizar:

  * Ataques que no requieren conocimientos técnicos
    * Ingeniería social
  * Ataques a servicios online de forma activa
    * Ataques de diccionario
    * Ataques por fuerza bruta
  * Ataques a servicios online de forma pasiva
    * Realizando capturas de tráfico de red
  * Ataques offline
    * Cracking de contraseñas

<a name="online"></a>

### Ataques contra servicios online de forma activa 

En los ataques contra servicios que esten online vamos a ir probando credenciales hasta que demos con una que sea válida. Normalmente intentaremos conseguir algún usuario en la fase de recopilación de información y después intentar adivinar la contraseña usando un diccionario o por fuerza bruta. Aunque no se recomienda, si no encontrasemos un usuario válido, también podríamos realizar un ataque de diccionario o fuerza bruta para averiguar el usuario. Como vemos los modos de atacar son:

  * Fuerza bruta
  * Diccionario
  
El ataque por fuerza bruta prueba todas las [permutaciones](https://es.wikipedia.org/wiki/Permutación) posibles de los símbolos que le indiquemos hasta que de con el usuario o contraseña. Es un método muy lento a no ser que la contraseña sea muy corta. En un ataque de diccionario creamos una lista (diccionario) de palabras que creemos que pueden ser la contraseña. Existen muchos servicios online contra los que podemos lanzar un ataque. Algunos de ellos son:

  * SSH
  * Aplicaciones Web
  
#### Ataque de diccionario contra SSH usando patator

Tras descubrir un servicio SSH en la fase de recopilación de información:

<img src="img/credenciales_ssh_nmap.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a ver como realizar un ataque de diccionario con el programa [patator](https://tools.kali.org/password-attacks/patator). Este programa ya le utilizamos para lanzar ataques de diccionario con el fin de enumerar los usuarios a través de SMTP. Para saber el módulo de patator que debemos utilizar, escribimos *patator* para listar los módulos y ver cuál es el módulo para ssh:

<img src="img/credenciales_ssh_patator_1.png" style="float:left">
<div style="clear:both"></div><br>

y una vez localizado el módulo, para saber como usarlo, escribimos *patator* seguido del nombre del módulo. En ese caso *patator ssh_login*:

<img src="img/credenciales_ssh_patator_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que para poder lanzar el ataque debemos indicarle:

  * **host** => máquina a la que queremos atacar
  * **user** => el usuario para el que vamos a intentar adivinar la contraseña
  * **password** => para indicarle el listado (diccionario) de posibles contraseñas

Creamos una lista (diccionario) de posibles contraseñas usando cualquier editor de texto. En este caso usamos el editor vim:

<img src="img/credenciales_ssh_patator_3.png" style="float:left">
<div style="clear:both"></div><br>

para crear la lista:

<img src="img/credenciales_ssh_patator_4.png" style="float:left">
<div style="clear:both"></div><br>

y lanzamos patator usando el usuario *msfadmin* que descubrimos en la fase de recopilación de información. Sabiendo que la máquina metasploitable 2 esta en la dirección (192.168.119.8), el commando es:

*patator ssh_login host=192.168.119.8 user=msfadmin password=FILE0 0=contraseñas_a_comprobar.txt*

<img src="img/credenciales_ssh_patator_5.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que *patator* va comprobando todas las contraseñas y nos informa de la que funciona. En este caso *msfadmin*. Como hicimos al enumerar a través del protocolo SMTP, podemos ignorar las contraseñas no válidas usar la opción *-x ignore:*. En este caso usariamos *-x ignore:mesg='Authentication failed.'*. El comando completo es:

*patator ssh_login host=192.168.119.8 user=msfadmin password=FILE0 0=contraseñas_a_comprobar.txt -x ignore:mesg='Authentication failed.'*

<img src="img/credenciales_ssh_patator_6.png" style="float:left">
<div style="clear:both"></div><br>

y como vemos sólo nos mostraría la contraseña válida. Podemos comprobar que la contraseña es válida conectandonos por ssh a la máquina usando el usuario *msfadmin*:

<img src="img/credenciales_ssh_patator_7.png" style="float:left">
<div style="clear:both"></div><br>

y la contraseña *msfadmin*:

<img src="img/credenciales_ssh_patator_8.png" style="float:left">
<div style="clear:both"></div><br>

Una vez conectados podemos ejecutar algún comando como *whoami*:

<img src="img/credenciales_ssh_patator_9.png" style="float:left">
<div style="clear:both"></div><br>

#### Ataque de diccionario contra formularios web usando Burp Suite

Tras descubrir un servidor web en la fase de recopilación de información:

<img src="img/credenciales_http_nmap.png" style="float:left">
<div style="clear:both"></div><br>

Accedemos a el servicio usando un navegador web como Firefox y hacemos click sobre *DVWA*:

<img src="img/credenciales_http_dvwa_1.png" style="float:left">
<div style="clear:both"></div><br>

Nos logamos en la aplicación con:

  * **Username:** admin
  * **Password:** password

<img src="img/credenciales_http_dvwa_2.png" style="float:left">
<div style="clear:both"></div><br>

A continuación hacemos click en el botón *DVWA Security* de la barra lateral izquierda:

<img src="img/credenciales_http_dvwa_3.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos *Low* en el desplegable:

<img src="img/credenciales_http_dvwa_4.png" style="float:left">
<div style="clear:both"></div><br>

y pulsamos el botón *Submit*:

<img src="img/credenciales_http_dvwa_5.png" style="float:left">
<div style="clear:both"></div><br>

Ahora hacemos click en el botón *Brute Force* en la barra lateral izquierda:

<img src="img/credenciales_http_dvwa_6.png" style="float:left">
<div style="clear:both"></div><br>

Para ver el formulario sobre el que haremos el ataque de diccionario:

<img src="img/credenciales_http_dvwa_7.png" style="float:left">
<div style="clear:both"></div><br>

Una vez en la página del formulario, vamos a seguir los mismos pasos que en el capítulo de recopilación de información para activar [Burp Suite](https://portswigger.net/burp/), para ello, desplegamos el menú de opciones en Firefox y seleccionamos *Preferences*:

<img src="img/credenciales_http_burp_1.png" style="float:left">
<div style="clear:both"></div><br>

A continuación pulsamos el botón *Settings...* dentro de la pestaña Network de la sección Advanced:

<img src="img/credenciales_http_burp_2.png" style="float:left">
<div style="clear:both"></div><br>

Cumplimentamos las opciones como se ve en la siguiente imagen y pulsamos el botón *OK*:

<img src="img/credenciales_http_burp_3.png" style="float:left">
<div style="clear:both"></div><br>

Después ejecutamos *burp*:

<img src="img/credenciales_http_burp_4.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos el botón *Next* para continuar:

<img src="img/credenciales_http_burp_5.png" style="float:left">
<div style="clear:both"></div><br>

Finalmente pulsamos el botón *Start Burp* para iniciar Burp Suite:

<img src="img/credenciales_http_burp_6.png" style="float:left">
<div style="clear:both"></div><br>

Cuando se inicie burp, pulsamos en la pestaña *Proxy* y después en la pestaña *Options*. En las opciones activamos "Incercept responses based on the following rules" para que que el proxy de burp intercepte las respuestas http del servidor antes de mostrarlas en el navegador:

<img src="img/credenciales_http_burp_7.png" style="float:left">
<div style="clear:both"></div><br>

Después hacemos click en la pestaña *Intercept* y nos aseguramos que el botón tiene el texto:

*Intercept is on*

<img src="img/credenciales_http_burp_8.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos a Firefox. Metemos el usuario y contraseña que queramos en formulario y pulsamos el botón *Login*:

<img src="img/credenciales_http_burp_9.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos a burp y veremos que la petición http que ha hecho el navegador ha sido capturada. Pulsamos el botón *Forward* para continuar:

<img src="img/credenciales_http_burp_10.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos a hacer click en el botón *Forward*:

<img src="img/credenciales_http_burp_11.png" style="float:left">
<div style="clear:both"></div><br>

para que la respuesta del servidor llegue a nuestro navegador:

<img src="img/credenciales_http_burp_12.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que las credenciales no existen -a no ser que hayamos tenido mucha suerte- y nos aparece el mensaje de login failed. Vamos a usar burp intruder para lanzar un ataque de diccionario contra el usuario y la contraseña. Para ello, seleccionamos la petición HTTP que nos ha dado la respuesta del servidor en la pestaña *HTTP history* y hacemos click con el botón derecho del ratón para desplegar el menú de opciones. Seleccionamos la opción *Send to Intruder*:

<img src="img/credenciales_http_burp_13.png" style="float:left">
<div style="clear:both"></div><br>

Hacemos click en la Pestaña Intruder y después en la pestaña *Positions*:

<img src="img/credenciales_http_burp_14.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos *Cluster bomb* en el menú desplegable *Attack Type*:

<img src="img/credenciales_http_burp_15.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos el botón *Clear*:

<img src="img/credenciales_http_burp_16.png" style="float:left">
<div style="clear:both"></div><br>

para desmarcar todos los valores de los parámetros marcados en naranja:

<img src="img/credenciales_http_burp_17.png" style="float:left">
<div style="clear:both"></div><br>

Ahora seleccionaremos los valores de los parámetros sobre los que queremos hacer fuerza bruta. Hacemos doble click sobre el valor del parámetro username y pulsamos el botón *Add*:

<img src="img/credenciales_http_burp_18.png" style="float:left">
<div style="clear:both"></div><br>

A continuación, hacemos doble click en el parámetro password y colvemos a pulsar el botón *Add*:

<img src="img/credenciales_http_burp_19.png" style="float:left">
<div style="clear:both"></div><br>

Una vez añadidos los parámetros, pulsamos el botón *Payloads*:

<img src="img/credenciales_http_burp_20.png" style="float:left">
<div style="clear:both"></div><br>

Dejamos las opciones que vienen por defecto en la sección * Payload Sets*:

  * **Payload set:** 1
  * **Payload type:** Simple list
  
y en la sección *Payload Options [Simple list]* vamos añadiendo usuarios a la lista. Escribimos un usuario y hacemos click en el botón *Add*:

<img src="img/credenciales_http_burp_21.png" style="float:left">
<div style="clear:both"></div><br>

Hacemos lo mismo por cada usuario que queramos añadir. Por ejemplo, añadimos otro usuario que se llame admin:

<img src="img/credenciales_http_burp_22.png" style="float:left">
<div style="clear:both"></div><br>

Una vez especificados los usuarios que queremos probar, seleccionamos el número 2 en el menú desplegable *Payload type:* para indicar las contraseñas que queremos probar para cada usuario que hemos definido en el punto anterior:

<img src="img/credenciales_http_burp_23.png" style="float:left">
<div style="clear:both"></div><br>

En este caso, en vez ir añadiendo las contraseñas que queremos probar una a una, utilizaremos un fichero (diccionario) con las contraseñas a probar. Para ello, seleccionamos *Runtime file* del menú desplegable *Payload type*:

<img src="img/credenciales_http_burp_24.png" style="float:left">
<div style="clear:both"></div><br>

Ahora pulsaremos el botón *Select File...* para seleccionar el fichero (diccionario) donde tenemos la lista de contraseñas:

<img src="img/credenciales_http_burp_25.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos el fichero y pulsamos el botón *Open*:

<img src="img/credenciales_http_burp_26.png" style="float:left">
<div style="clear:both"></div><br>

Una vez seleccionado el fichero, pulsamos sobre la pestaña *Options*:

<img src="img/credenciales_http_burp_27.png" style="float:left">
<div style="clear:both"></div><br>

y en la sección *Grep - Extract*  activamos la opción *Extract the following from responses:* y pulsamos el botón *Add*:

<img src="img/credenciales_http_burp_28.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos el texto de error que muestra cuando las credenciales son incorrectas y pulsamos el botón *OK*:

<img src="img/credenciales_http_burp_29.png" style="float:left">
<div style="clear:both"></div><br>

Ya podemos lanzar el ataque de diccionario contra el usuario y la contraseña pulsando el botón *Start Attack*:

<img src="img/credenciales_http_burp_30.png" style="float:left">
<div style="clear:both"></div><br>

Si tenemos la versión gratuita de burp nos aparecerá el siguiente mensaje. Pulsamos el botón *OK*:

<img src="img/credenciales_http_burp_31.png" style="float:left">
<div style="clear:both"></div><br>

y vemos como va probando todas las contraseñas para cada uno de los usuarios y, en la única donde no hay mensaje de error, es la que nos indica el usuario y contraseña correcto:

<img src="img/credenciales_http_burp_32.png" style="float:left">
<div style="clear:both"></div><br>